In [3]:
# For making the example's imports work in this folder.
#import sys, os
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))

In [4]:
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
from examples._1_simple_mnist.model import MLP
from examples._1_simple_mnist.aggregate import Classifier, ClassifierSettings
from examples._1_simple_mnist.repository import Classifiers
from examples._1_simple_mnist.dataset import Digits
from examples._1_simple_mnist.handlers import subscriber

In [5]:
model = MLP(input_size=28*28, hidden_size=128, output_size=10, p=0.5)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
dataloaders = [
    ('train', DataLoader(Digits(train=True, normalize=True), batch_size=32, shuffle=True)),
    ('test', DataLoader(Digits(train=False, normalize=True), batch_size=32, shuffle=False))
]

In [6]:
from torchsystem import Callbacks
from torchsystem.metrics.average import Accuracy, Loss

settings = ClassifierSettings()
classifier = Classifier(model, criterion, optimizer, settings)
classifier.bind(subscriber)
repository = Classifiers()
callbacks = Callbacks(Loss(), Accuracy())

In [7]:
for epoch in range(2):
    for phase, loader in dataloaders:
        classifier.phase = phase
        classifier.fit(loader, callbacks) if phase == 'train' else classifier.evaluate(loader, callbacks)
        for callback in callbacks:
            classifier.publish({'epoch': epoch, 'phase': phase, 'metric': callback.name, 'average': callback.average.value}, topic='results')
            callback.reset()
repository.store(classifier)
classifier.publish('End of training-----------------', topic='end')

Batch: 300, Metric: loss, Value: 0.9219042062759399, Phase: train
Batch: 300, Metric: accuracy, Value: 0.6980208333333338, Phase: train
Batch: 600, Metric: loss, Value: 0.7127916216850281, Phase: train
Batch: 600, Metric: accuracy, Value: 0.7699479166666672, Phase: train
Batch: 900, Metric: loss, Value: 0.6215074062347412, Phase: train
Batch: 900, Metric: accuracy, Value: 0.8027083333333336, Phase: train
Batch: 1200, Metric: loss, Value: 0.563820481300354, Phase: train
Batch: 1200, Metric: accuracy, Value: 0.8233593750000003, Phase: train
Batch: 1500, Metric: loss, Value: 0.5233115553855896, Phase: train
Batch: 1500, Metric: accuracy, Value: 0.837291666666667, Phase: train
Batch: 1800, Metric: loss, Value: 0.4930669069290161, Phase: train
Batch: 1800, Metric: accuracy, Value: 0.8477604166666669, Phase: train
Results {'epoch': 0, 'phase': 'train', 'metric': 'loss', 'average': tensor(0.4876, grad_fn=<DivBackward0>)}
Results {'epoch': 0, 'phase': 'train', 'metric': 'accuracy', 'average': 